In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) 

train_dataset = dsets.MNIST(
    root='./data',
    train=True,
    transform=trans,
    download=True
)
 
test_dataset = dsets.MNIST(
    root='./data',
    train=False,
    transform=trans
)

In [4]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
 
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [5]:
class ConvNet(nn.Module): 
    def __init__(self): 
        super(ConvNet, self).__init__() 
        self.layer1 = nn.Sequential( nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
        self.layer2 = nn.Sequential( nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
        self.drop_out = nn.Dropout() 
        self.fc1 = nn.Linear(7 * 7 * 64, 1000) 
        self.fc2 = nn.Linear(1000, 10)
    
    def forward(self, x): 
        out = self.layer1(x) 
        out = self.layer2(out) 
        out = out.reshape(out.size(0), -1) 
        out = self.drop_out(out) 
        out = self.fc1(out) 
        out = self.fc2(out) 
        return out

In [6]:
model = ConvNet()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
%%time

total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/5], Step [100/600], Loss: 0.1144, Accuracy: 95.00%
Epoch [1/5], Step [200/600], Loss: 0.1908, Accuracy: 94.00%
Epoch [1/5], Step [300/600], Loss: 0.0208, Accuracy: 100.00%
Epoch [1/5], Step [400/600], Loss: 0.0152, Accuracy: 100.00%
Epoch [1/5], Step [500/600], Loss: 0.0852, Accuracy: 97.00%
Epoch [1/5], Step [600/600], Loss: 0.0886, Accuracy: 98.00%
Epoch [2/5], Step [100/600], Loss: 0.1330, Accuracy: 95.00%
Epoch [2/5], Step [200/600], Loss: 0.0661, Accuracy: 98.00%
Epoch [2/5], Step [300/600], Loss: 0.0876, Accuracy: 96.00%
Epoch [2/5], Step [400/600], Loss: 0.0367, Accuracy: 98.00%
Epoch [2/5], Step [500/600], Loss: 0.0297, Accuracy: 100.00%
Epoch [2/5], Step [600/600], Loss: 0.0358, Accuracy: 99.00%
Epoch [3/5], Step [100/600], Loss: 0.1358, Accuracy: 98.00%
Epoch [3/5], Step [200/600], Loss: 0.0983, Accuracy: 97.00%
Epoch [3/5], Step [300/600], Loss: 0.1408, Accuracy: 97.00%
Epoch [3/5], Step [400/600], Loss: 0.1018, Accuracy: 97.00%
Epoch [3/5], Step [500/600], Loss: 0.

In [27]:
%%time
# Inference time on unoptimized model

for i, (images, labels) in enumerate(test_loader):
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)  # Выбор лучшего класса из выходных данных: класс с лучшим счетом
    total += labels.size(0)                    # Увеличиваем суммарный счет
    correct += (predicted == labels).sum()     # Увеличиваем корректный счет
       
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 98 %
CPU times: user 9.93 s, sys: 109 ms, total: 10 s
Wall time: 3.79 s


In [21]:
# Saving the model
import torch.onnx
import torchvision

dummy_input = torch.randn(batch_size, 1, 28, 28)
torch.onnx.export(model, dummy_input, "cnn.onnx")

In [25]:
# Extacting model after inference
import openvino.inference_engine
from openvino.inference_engine import IENetwork, IEPlugin

plugin_dir = None
model_xml = '/home/irek/Works/3kurs/Science/Autoencoder/cnn.xml'
model_bin = '/home/irek/Works/3kurs/Science/Autoencoder/cnn.bin'
plugin = IEPlugin("CPU", plugin_dirs=plugin_dir)
net_vino = IENetwork(model=model_xml, weights=model_bin)
exec_net = plugin.load(network = net_vino)
input_blob = next(iter(net_vino.inputs))
output_blob = next(iter(net_vino.outputs))

In [26]:
%%time
# Inference on optimized model

for i, (images, labels) in enumerate(test_loader):
    res = exec_net.infer(inputs = {input_blob: images})

CPU times: user 8.48 s, sys: 60.1 ms, total: 8.54 s
Wall time: 5.5 s
